# QBoost

このノートでは QBoost と呼ばれるアニーリングマシンを用いた機械学習手法を紹介します。

## 目次

- QBoostとは
- データセットの生成
- データセットの分割
- 弱分類器の多数決による分類
- Amplify AEを用いた改良
 - ノイズとなる分類器を除去する手法による結果
 - ブースティングの思想を取り入れたQBoostによる結果
- まとめ

前提知識として、教師あり学習、教師なし学習、分類、学習、テスト、バリデーションなどの単語の意味を知っていることを仮定します。


## QBoostとは

QBoostは機械学習の手法の一つで、アンサンブル学習（Ensemble Learning）といわれる分野に分類されます。より具体的には、ブースティングの思想を取り入れたバギングを行なっています。

アンサンブル学習は別々に学習させた分類器（Classifier）を組み合わせて推論させる学習方法のことで、独立に学習した分類器・回帰モデルの出力の多数決もしくは平均を取る手法（バギング）、過去に学習した分類器の出力を元に、学習済みの分類器の弱点を補うように新しい分類器を学習する手法（ブースティング）などがあります。

理論的には、バギングを用いることで予測のブレ（分散、バリアンス）を軽減し、ブースティングにより予測のズレ（偏り、バイアス）を減少させることが期待されます。バイアスとバリアンスがともに小さいモデルが理想ですが、この二つは互いにトレードオフの関係にあることが知られています。

アンサンブル手法の最も単純な実装は、各分類器の出力の多数決をとるバギングと呼ばれる手法です。しかし、単純に多数決を取るだけではいわゆる「良くない」分類器が紛れ込んでいた場合に、思ったように精度が上がらないなどの問題があります。QBoost はそのような問題の解決を試みる方法です。以下では、扱う問題の説明から始め、実際に Amplify を用いて単純な多数決よりも精度が向上することを確認します。

## データセットの生成

まずはQBoostを評価するためのデータセットを作成します。

今回使用するデータセットは、`scikit-learn` に含まれている `iris_dataset` です。`iris_dataset`とは、計3種類のアヤメとそれに対応する4つの特徴量（花弁の長さ・幅、ガクの長さ・幅）が含まれたデータセットであり、この特徴量を用いてアヤメの分類を行うことが今回の目標です。つまり、教師あり学習による3クラス分類に取り組みます。このデータセットには150個分のアヤメのデータが含まれています。

以下が各種類のアヤメの写真です（wikipediaから引用）。パッと見ただけではほとんど違いがわからないと思います。

![](./figures/iris_pictures.png)

次のように `iris_dataset` を読み込みます。

In [ ]:
from sklearn.datasets import load_iris
import numpy as np

iris_dataset = load_iris()
data, label = np.array(iris_dataset["data"]), np.array(iris_dataset["target"])

## データセットの分割

次にデータセットを学習データとテストデータに分割します。ここでは適当に60%を学習データ、20%をテストデータ、残りの20%をバリデーションデータとしています。データセットの分割には`scikit-learn`の`train_test_split`関数を使用します。クラスごとのデータ数に偏りが少なくなるように分割できています。

後ほど詳しく説明しますが、今回は、各クラスに対してそのクラスに属しているか否かを判別する2値分類器を用いて3クラス分類を行うため、それに適した形になるようにラベルを二次元配列で整形します。

In [ ]:
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt


num_classes = len(set(label))

# データセットに含まれるラベルが2種類なら分類器は１クラス分あれば良い
if num_classes == 2:
    num_classes = 1

# 全体の 20% をテストデータに分割する
X_train, X_test, y_train_tmp, y_test_tmp = train_test_split(
    data, label, test_size=0.2, random_state=0
)

# 残りの 25% (全体の20%) をバリデーションデータに分割する
X_train, X_valid, y_train_tmp, y_valid_tmp = train_test_split(
    X_train, y_train_tmp, test_size=0.25, random_state=0
)

# 分類数xデータ数の二次元配列を作成しデータに対応した正解ラベルを表す
# 一致していたら1, 不一致の場合 -1
y_train = np.full(shape=(num_classes, len(X_train)), fill_value=-1)
y_test = np.full(shape=(num_classes, len(X_test)), fill_value=-1)
y_valid = np.full(shape=(num_classes, len(X_valid)), fill_value=-1)


def update_label(y, tmp):
    for i, label in enumerate(tmp):
        y[label, i] = 1


# 正解ラベルを配列に埋め込む
update_label(y_train, y_train_tmp)
update_label(y_test, y_test_tmp)
update_label(y_valid, y_valid_tmp)

# データ数を表示
print(f"train data: {len(X_train)}")
for cls in range(num_classes):
    print("train", f"label {cls}:", np.sum(y_train_tmp == cls))
print(f"test data: {len(X_test)}")
for cls in range(num_classes):
    print("test", f"label {cls}:", np.sum(y_test_tmp == cls))
print(f"validation data: {len(X_valid)}")
for cls in range(num_classes):
    print("validation", f"label {cls}:", np.sum(y_valid_tmp == cls))

In [ ]:
# テストデータの可視化
from itertools import combinations

# 各特徴量の名前
feature_names = iris_dataset.feature_names


def plot_features(X, Y, fig=None):
    if fig is None:
        fig = plt.figure(figsize=(12, 8))

    # 二次元のグラフを作りたいので特徴量の組み合わせを作る
    for i, (x, y) in enumerate(combinations(range(4), 2)):
        # サブグラフ
        plt.subplot(2, 3, i + 1)
        # 各品種はマーカーの色や形を変える
        for t, marker, c in zip(range(3), ">ox", "rgb"):
            plt.scatter(
                X[Y == t, x],
                X[Y == t, y],
                marker=marker,
                c=c,
                label=feature_names[t],
            )
            plt.xlabel(feature_names[x])
            plt.ylabel(feature_names[y])
    plt.autoscale()
    plt.grid()
    plt.legend()

    return fig


test_original = plot_features(X_test, y_test_tmp)

## 弱分類器の作成

次に、アンサンブル学習で使用する弱分類器を作成します。今回は弱分類器として、深さ1の決定木(Decision Tree)を各クラスごとに100個、合計で300個使用します。余談ですが、深さ1の決定木は決定株とも呼ばれます。

決定木の内部実装に関しては今回の本質ではないため、scikit-learnパッケージを使用しますが、今回は各分類器 ごとに学習データの中から決められたサンプル数 (`num_samples=5`) だけ取り出して学習させます。

In [ ]:
from sklearn.tree import DecisionTreeClassifier as DTC

# 乱数シードを固定
seed = 0
np.random.seed(seed)

# 分類器を二次元リストで作成
num_classifiers = 100
num_samples = 5
depth = 1
classifiers = [
    [DTC(splitter="random", max_depth=depth) for _ in range(num_classes)]
    for _ in range(num_classifiers)
]

# 決定木を学習
all_indices = np.arange(X_train.shape[0])
for classifier in classifiers:
    for i in range(num_classes):
        sample_indices = np.random.choice(all_indices, num_samples)
        classifier[i].fit(X=X_train[sample_indices], y=y_train[i, sample_indices])

print(f"number of classifiers {num_classifiers * num_classes}")

### 多数決による分類

まずはアンサンブル学習を分類問題に適用する際の基本である、多数決による結果を計算します。全ての弱学習器の結果の最頻値を分類器全体の出力と解釈します。このように、独立に学習した弱分類器を用いたアンサンブル学習を一般にバギングと呼びます。特に、独立に学習した決定木をバギングによってアンサンブルする手法をランダムフォレストと呼びます。

今回はラベルが±1をとる2値分類を扱っているため、実装上は分類器 $i$ の予測ラベルを $y_i\in\{-1, 1\}$ として、$\text{sign}\left(\sum_{i=1}^{\#\textit{classifiers}}y_i\right)$ を計算することで多数決を実現しています。
問題自体が非常に簡単な部類のため、この時点でもかなり良い精度が出ています。

用意した分類器の精度は、以下のようにして確認します。
1. ラベルiに対応する分類器のみを用いて、正しく分類できているか確かめる(i=0, 1, 2)
2. 用意した分類器を全て使用し、正しく分類できているか確かめる

2に関しては、データがラベルiである、と予測した分類器が最も多いラベルを分類器全体の出力とみなして正解率を計算します。

In [ ]:
from sklearn import metrics
import scipy.stats as stats

results = {
    "多数決": dict(),
    "QBoost(step 1)": dict(),
    "QBoost(step 2)": dict(),
}  # 結果保存用

# テストデータとバリデーションデータに対して推論
test_predictions_of_classifiers = np.array(
    [
        [classifier[i].predict(X_test) for i in range(num_classes)]
        for classifier in classifiers
    ]
)
valid_predictions_of_classifiers = np.array(
    [
        [classifier[i].predict(X_valid) for i in range(num_classes)]
        for classifier in classifiers
    ]
)

# 多数決
for l in range(num_classes):
    predictions_vote = np.sign(np.sum(test_predictions_of_classifiers[:, l, :], axis=0))

    # Calculate accuracy
    accuracy = metrics.accuracy_score(y_true=y_test[l], y_pred=predictions_vote)

    # クラスごとの正解率を表示
    print(f"Majority vote of weak classifiers class {l}: {accuracy*100} %")
    results["多数決"][f"class {l}"] = accuracy * 100

In [ ]:
# 全てのラベルに対応する分類器の出力を統合した分類結果の正解率を取得
if num_classes > 2:
    tmp = np.sum(test_predictions_of_classifiers, axis=0)
    m_vote = np.argmax(tmp, axis=0)
    accuracy = metrics.accuracy_score(y_true=y_test_tmp, y_pred=m_vote)
    print(f"Majority vote of weak classifiers: {accuracy*100} %")
    results["多数決"]["Total"] = accuracy * 100
    results["多数決"]["num_classifiers"] = num_classes * num_classifiers

## Amplify を用いた QBoost の実装

このセクションでは、上記の単純な多数決の結果の改善を目指して、Amplify を用いた QBoost の実装について解説します。1段階目としてノイズとなる余計な分類器を除外する手法を紹介し、二段階目に弱分類器の修正を行う手法 (QBoost) を解説します。

### 1段階目：最適な分類器の組み合わせを求める

まず、QBoostの導入となる単純な手法を紹介します。
この手法のアイデアは、全ての分類器の中から最も高い精度が得られる組み合わせを求めてそれを使用するというものです。つまり、ノイズとなる余計な分類器を除外します。そこで「各分類器に対して、それを使用するか/しないか」をバイナリ変数で表します。

まずは2クラス分類に対する定式化を行います。

2クラス分類において、いくつかの弱分類器を取り除いたあと残ったものの多数決によって分類を行うとき、その結果は

$$ H(x) = {\rm sign}\left(\sum_{i=1}^{C}s_i h_i(x)\right)$$

となります。ここで、$C$は分類器の数、$h_i(x) \in \{-1, 1\}$は弱分類器、$s_i \in \{0, 1\}$はバイナリ変数です。

今回の目的を達成するために一番自然な目的関数は、誤分類の個数を数え、それを最小化すること、つまり
$$\left(1-\textit{label}_x\times{\rm sign}\left(\sum_{i=1}^{C}s_i h_i(x_j)\right)\right)/2$$
を最小化することです。
しかし、これは符号関数が含まれ非凸なので最適化が難しいです。
そこで、代わりにその上界である
$$\left(\textit{label}_x\times\sum_{i=1}^{C}s_i h_i(x)-1\right)^2$$
を最小化することで、目的を達成しようと試みることにします。

一方で、使用する弱学習器が多すぎる場合、過学習によりテストデータの正解率が低くなってしまうことがあります。
それを防ぐため、使用する弱分類器の数$\sum_{i=1}^Cs_i$をペナルティ項として上の式に重みをつけて足し合わせます。

したがって、2クラス分類においては、バイナリ変数を最適化するために最小化すべき関数は以下のようになります。

$$ s_{\textit{min}} = \mathop{\rm arg~min}\limits_{s} \sum_{j=1}^{N_{\textit{train}}}\left(y_j
\sum_{i=1}^{C}s_i h_i(x_j)-1\right)^2 + P\sum_{i=1}^{C}s_i$$

$N_{\textit{train}}$は学習データの数、$y_j \in \{-1, 1\}$はデータ$x_j$に対するラベル、$P$ は使用する弱分類器の数に対するペナルティ係数です。この値を大きく設定するほど、使用する弱学習器が少なくなります。

次に、クラスごとに2値分類を行った結果を統合して多クラス分類を行うことを考えます。 
今回の例では$3(\textrm{クラス数})\times100(\textrm{クラスごとの分類器数})$、計300個の弱分類器を使用しているため、300個のバイナリ変数を定義し、
以下の関数を最小化します。
$$ s_{min} = \mathop{\rm arg~min}\limits_{s} \sum_{j=1}^{N_{train}}\sum_{l=0}^{\#labels-1}\left(y_j
\sum_{i=1}^{C}s_{li} h_{li}(x_j)-1\right)^2 + P\sum_{l=0}^{\#labels-1}\sum_{i=1}^{C}s_{li}$$

これは、2クラス分類で見たのと同様に、それぞれのラベルに対し、そのラベルの分類精度を向上させるような弱分類器の組み合わせを求めることを目的としています。

このようにして使用する弱分類器を決定したあと、次のように新しい分類器$G(x)$を定義します。

$$ G(x) = {\rm arg~max}_l\left\{\sum_{i=1}^{C}s_{li} h_{li} \mid l=0,1,\dots\#\textit{labels}-1\right\} $$

(今回は、$\#\textit{labels}=3$に相当します。)
$C$は分類器の数、$h_{li}(x) \in \{-1, 1\}$は弱分類器、$s_{li} \in \{0, 1\}$はバイナリ変数です。

$ \displaystyle\sum_{i=1}^{C}s_{li} h_{li}(x) $ は、ラベル$l$に対応する分類器のなかで、データ$x$に対応するラベルが$l$であると予測した分類器の数の多さを表現しているため、この値が最も大きいラベルを予測として出力することで、多数決を達成することができます。

参考文献：[Training a Binary Classifier with the
Quantum Adiabatic Algorithm](https://arxiv.org/abs/0811.0416)

まず、前処理として、学習データに対する各分類器の出力を求め、Amplify AEを用いた最適化のための変数を作成します。

In [ ]:
from amplify import BinarySymbolGenerator


def preprocess(classifiers):
    """最適化に必要な変数の定義と学習データに対する分類器の出力を計算"""
    # Prepare spins
    gen = BinarySymbolGenerator()
    spins = gen.array(shape=(num_classifiers, num_classes))

    # Obtain predictions for train data
    train_predictions_of_classfiers = np.array(
        [
            [classifier[i].predict(X_train) for i in range(num_classes)]
            for classifier in classifiers
        ]
    )
    return spins, train_predictions_of_classfiers

続いて、目的関数を作成します。

In [ ]:
from amplify import sum_poly


def createQUBO(spins, penalty, train_predictions_of_classifiers):
    """QUBOとして最適化する関数を定義"""
    # 各データ、各クラスラベルに対応する分類器について、分類器の総和をとる。
    tmp = (np.expand_dims(spins, -1) * train_predictions_of_classifiers).sum(0)
    tmp = (y_train * tmp - 1) ** 2
    f = tmp.sum()

    f += penalty * spins.sum()
    return f

次に、最適化と結果の取り出しを行う関数を作成します。 
最適化にはアニーリングマシン（Fixstars Amplify AE）が使用されます。

In [ ]:
from amplify import Solver, decode_solution
from amplify.client import FixstarsClient


def solve(f, spins, solver=None):
    """Amplify AEを用いて最適化"""
    if solver is None:
        client = FixstarsClient()
        client.parameters.timeout = 1000
        solver = Solver(client)
    # Solve QUBO formulation
    result = solver.solve(f)
    solution = decode_solution(spins, result[0].values)
    # 最適化結果の取り出し
    use_indices = [
        np.where(solution[:, l] == 1)[0].tolist() for l in range(num_classes)
    ]
    return use_indices

最後に、最適化の結果得られた分類器の正解率を計算します。

In [ ]:
def compute_accuracy(use_indices, predictions_of_classifiers, y_true):
    """与えられたインデックスに対応する分類器による推論結果と正解率を計算"""
    model_predictions = []
    accuracy_each_class = []
    ACCEPT = True
    for i in range(num_classes):
        if (
            len(use_indices[i]) == 0
        ):  # 全く使われないクラスの分類器がある場合、結果を採用しない
            ACCEPT = False
            break
        model_predictions.append(
            np.sum(predictions_of_classifiers[use_indices[i]][:, i, :], axis=0)
        )
        accuracy = metrics.accuracy_score(
            y_true=y_true[i], y_pred=list(map(np.sign, model_predictions[i]))
        )
        accuracy_each_class.append(accuracy)
    return np.array(model_predictions), np.array(accuracy_each_class), ACCEPT

次のセルを実行すると、上記の一連の操作が実行されます。

In [ ]:
penalty = (
    len(X_train) / num_classes / num_classifiers
)  # ペナルティ項の係数。今回は決め打ち

# Obtain predictions for train data　and prepare spins
spins, train_predictions_of_classifiers = preprocess(classifiers)
# Create QUBO formulation
f = createQUBO(spins, penalty, train_predictions_of_classifiers)
use_indices = solve(f, spins)
# バリデーションデータに対する正解率を計算
valid_pred, valid_acc, _ = compute_accuracy(
    use_indices, valid_predictions_of_classifiers, y_valid
)
# テストデータに対する正解率を計算
test_pred, test_acc, _ = compute_accuracy(
    use_indices, test_predictions_of_classifiers, y_test
)

最後に、結果を使用して精度の計算を行います。QBoostの結果が単純な多数決の結果を超えていることを確認できます。

In [ ]:
for i in range(num_classes):
    print(f"QBoost(step 1) class {i}: {test_acc[i]*100} %")  # クラスごとの正解率を表示
    results["QBoost(step 1)"][f"class {i}"] = test_acc[i] * 100

results["QBoost(step 1)"]["num_classifiers"] = sum(
    len(use_indices[i]) for i in range(num_classes)
)  # 使用した分類器の数
print(results["QBoost(step 1)"]["num_classifiers"])

最終的な出力は以下の関数によって計算されます。
冒頭の単純な多数決と同様に、データがラベルiである、と予測した分類器が最も多いラベルを分類器全体の出力とみなして正解率を計算します。

In [ ]:
def model_output(model_predictions, valid_acc, y_true):
    """各ラベルに対応する分類器の予測の多数決をとり、最終的な出力と正解率を計算"""
    m_vote = np.argmax(model_predictions, axis=0)
    accuracy = metrics.accuracy_score(y_true=y_true, y_pred=m_vote)
    return np.array(m_vote), accuracy

In [ ]:
# Calculate accuracy
m_vote, accuracy = model_output(test_pred, valid_acc, y_test_tmp)
print(f"QBoost(step 1): {accuracy*100} %")
results["QBoost(step 1)"]["Total"] = accuracy * 100


### 2段階目：QBoost による分類器の修正

さて前置きが長くなりましたがQBoostを実行していきます。まずはQBoostがどのような方針でどのように定式化されるかを見ていきます。
 
QBoostでは
1. 全ての分類器の中から最も高い精度が得られる組み合わせを求めてそれを使用する (ノイズとなる余計な分類器を除外する)
2. 間違いが多い分類器をこれまでの分類結果に応じて更新する (ブースティングの思想) 

といった操作を繰り返します。 具体的には、以下のような操作となります。

まず、弱分類器からいくつかを選ぶ操作の結果として得られる新しい分類器を$H(x)$とし、先ほどと同様に定義します。
 
$$ H(x) = {\rm sign}\left(\sum_{i=1}^{C}s_i h_i(x)\right)$$


$C$は分類器の数、$h_i(x) \in \{-1, 1\}$は弱分類器、$s_i \in \{0, 1\}$はバイナリ変数です。

ここで、上の式のバイナリ変数$s$を最適化するために最小化すべき関数は以下のようになります。

$$ s_{min} = \mathop{\rm arg~min}\limits_{s} \sum_{j=1}^{N_{\textit{train}}}\left(y_j
\sum_{i=1}^{C}s_ih_i(x_j)-1\right)^2 + P\sum_{i=1}^{C}s_i$$

$N_{train}$は学習データの数、$y_j \in \{-1, 1\}$はデータ$x_j$に対するラベルで、$P$はバイナリ変数に対するペナルティ(定数)です。

次に、$H(x)$を定める過程で選ばれなかった弱分類器を重みづけした学習データで学習しなおすことにより更新します。
このとき、$j$番目の学習データの重み$d_j$は、分類器$H(x)$において$j$番目の学習データを誤分類する弱分類器が多いほど大きく重みづけるように、

$$
d_j \leftarrow d_j\times\left(y_j
\sum_{i=1}^{C}s_ih_i(x_j)-1\right)^2
$$

のようにして更新します。実際には、オーバーフローやアンダーフローを防ぐために、$\sum_{i=1}^Cd_j=1$となるように正規化して実装します。

次のループでは、このようにして更新された弱分類器と$H(x)$に含まれる弱分類器 (今回は合計100個あります) を用いて、再度弱分類器の最適な組み合わせを求め、選ばれなかった弱分類器を更新することになります。

以上が二値分類を行う場合のQBoostの大まかな流れです。
今回は、QBoostの多クラス分類への拡張として、一対多の二値分類を行う分類器を組み合わせることを考えます。

各クラスに対して、そのデータが自分のクラスに属しているかどうかを判別する2値分類器を用意し、その出力を総合して分類器全体の出力とします。
具体的には、クラスごとの分類器の出力からデータの予測クラスが一意に定まる場合はそれを出力し、そうでない場合はバリデーションデータに対する精度が最も高いクラスの分類器を優先します。

定式化に関して、先ほど紹介したシンプルな手法との差分は、パラメーター$d$を使用するか否かだけなので、多クラス拡張に関する説明・数式は省略します。

参考文献：[QBoost: Large Scale Classifier Training with
Adiabatic Quantum Optimization](http://proceedings.mlr.press/v25/neven12/neven12.pdf)

今回は、パラメーター$d$と弱学習器を反復ごと更新する必要があるため、その処理を行う関数を次のセルで定義します。

In [ ]:
def update_parameters(
    d_inner,
    best_idxs,
    use_idxs,
    classifiers,
    valid_predictions_of_classifiers,
    test_predictions_of_classifiers,
):
    """重み付けパラメーターdの更新と、最適化の結果使用しないとされた弱分類器の更新"""
    # 重み付けパラメーターdを更新
    for l in range(num_classes):
        d_broadcast = np.broadcast_to(
            d_inner[:, l], (len(best_idxs[l]), len(d_inner[:, l]))
        ).copy()
        tmp = (
            y_train[l]
            * np.sum(
                d_broadcast * train_predictions_of_classifiers[best_idxs[l]][:, l, :],
                axis=0,
            )
            - 1
        ) ** 2
        d_inner[:, l] *= tmp
        if np.sum(d_inner[:, l]) == 0:
            continue
        d_inner[:, l] /= np.sum(d_inner[:, l])

    # Update classifier dictionary
    # 選択されていない分類器を、データに重み付けをした上で更新する
    tmp = np.array(np.nonzero(use_idxs))
    for c in range(tmp.shape[-1]):
        i, l = tmp[:, c]
        classifier = classifiers[l][i]
        sample_indices = np.random.choice(np.arange(X_train.shape[0]), num_samples)
        classifier.fit(
            X=X_train[sample_indices],
            y=y_train[i, sample_indices],
            sample_weight=d_inner[sample_indices, i],
        )
        test_predictions_of_classifiers[l][i] = classifier.predict(X_test)
        valid_predictions_of_classifiers[l][i] = classifier.predict(X_valid)

    return (
        d_inner,
        classifiers,
        valid_predictions_of_classifiers,
        test_predictions_of_classifiers,
    )

続いて、ハイパーパラメーターの設定と、変数の初期化を行います。
ハイパーパラメーターとは、アルゴリズム実行前に人が設定する必要のあるパラメーターを指します。通常、ハイパーパラメーターはブラックボックス最適化手法を用いて（用意した候補の中で）最適なものが選択されますが、今回は適当に決め打ちしています。

In [ ]:
MAX_ITER = 1000  # 最大反復回数
EPS = 1e-8  # パラメータdのスケーリングチェック用

prev_acc = -1
best_acc = 0
best_idxs = None
best_T = -1
best_lam = None


# initialization
d_inner = np.full((len(X_train), num_classes), 1 / len(X_train))
T_inner = 0
lam = np.arange(0.001, penalty / 10, 0.01)  # ペナルティ項の係数
use_idxs = np.ones((num_classes, num_classifiers))

以上のことを踏まえると、QBoostは以下のコードによって実行されます。

In [ ]:
np.random.seed(seed)
for _iter in range(MAX_ITER):
    assert (
        abs(np.sum(d_inner, axis=0) - 1) < EPS
    ).all()  # 重み付け用のパラメーターがきちんとスケーリングされているかチェック
    for penalty in lam:
        print("iteration :", _iter, "lambda =", penalty)
        print(f"Best : {best_acc*100}%")

        # 変数の用意・学習データに対する推論結果の計算
        spins, train_predictions_of_classifiers = preprocess(classifiers)
        # QUBOモデルの作成
        f = createQUBO(spins, penalty, train_predictions_of_classifiers)
        # 求解・解の取り出し
        temporal_idxs = solve(f, spins)
        # クラスラベルごとの正解率を計算
        predictions_qboost, valid_acc, accept = compute_accuracy(
            temporal_idxs, valid_predictions_of_classifiers, y_valid
        )

        if not accept:
            # 最適化の結果、あるクラスの分類器を使わないことが最適とされた場合は処理を終了し次のループへ進む
            continue
        if num_classes > 2:  # 統合した結果を計算
            m_vote, accuracy = model_output(predictions_qboost, valid_acc, y_valid_tmp)

        if (
            accuracy >= best_acc
        ).all():  # 正解率が向上した場合、得られた解を最良解とする
            best_acc = accuracy
            best_idxs = temporal_idxs
            best_T = sum(len(temporal_idxs[i]) for i in range(num_classes))
            for i in range(num_classes):
                use_idxs[i, best_idxs[i]] = 0
                best_lam = penalty

    if (prev_acc >= best_acc).all():  # 最良解の更新が止まったタイミングで処理を終了する
        print(
            "Finish!",
            f"Best acculacy = {best_acc*100}%, number of classifiers = {best_T}, best lambda = {best_lam}",
        )
        break
    prev_acc = best_acc

    # 重みづけパラメーターと、未使用の分類器を更新
    (
        d_inner,
        classifiers,
        valid_predictions_of_classifiers,
        test_predictions_of_classifiers,
    ) = update_parameters(
        d_inner,
        best_idxs,
        use_idxs,
        classifiers,
        valid_predictions_of_classifiers,
        test_predictions_of_classifiers,
    )

続いて、最適化の結果を確認します。まずはクラスごとの正解率を確認します。

In [ ]:
valid_pred, valid_acc, _ = compute_accuracy(
    best_idxs, valid_predictions_of_classifiers, y_valid
)
test_pred, test_acc, _ = compute_accuracy(
    best_idxs, test_predictions_of_classifiers, y_test
)

for i in range(num_classes):
    print(f"QBoost class {i}: {test_acc[i]*100} %")
    results["QBoost(step 2)"][f"class {i}"] = test_acc[i] * 100
    # print("precision", metrics.precision_score(y_true=y_test[i], y_pred=predictions_qboost[i])*100)
    # print("recall", metrics.recall_score(y_true=y_test[i], y_pred=predictions_qboost[i])*100)

print(best_T)
results["QBoost(step 2)"]["num_classifiers"] = best_T

続いて、各クラスに対応する分類器の出力を統合した場合の正解率を確認します。

In [ ]:
if num_classes > 2:
    m_vote, accuracy = model_output(test_pred, valid_acc, y_test_tmp)
    print(f"QBoost(step 2): {accuracy*100} %")
    results["QBoost(step 2)"]["Total"] = accuracy * 100

※ 単純な問題を扱ったためQBoostでの結果が単純な多数決より悪くなる場合もありますが、ご了承ください。

## まとめ
ナイーブな手法、ノイズとなる分類器を除去する手法、QBoostの結果をまとめると以下のようになります。 
この結果から QBoost により分類精度を改善できていることがわかります。

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.T